In [ ]:
!pip install diffusers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.2 MB/s eta 0:00:00


## Example given by Hugging Face for clip score / LDM
https://huggingface.co/docs/diffusers/en/conceptual/evaluation

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

model_ckpt = "CompVis/stable-diffusion-v1-4"
sd_pipeline = StableDiffusionPipeline.from_pretrained(model_ckpt, torch_dtype=torch.float16).to("cuda")

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

(…)ature_extractor/preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

scheduler/scheduler_config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

safety_checker/config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

In [ ]:
sd_pipeline.safety_checker = None

# Can be set to 1~50 steps. LCM support fast inference even <= 4 steps. Recommend: 1~8 steps.
total_clip_score = 0
i = 0
stop = 10
for prompt in prompts:
  if i == stop:
    break
  num_inference_steps = 1

  image = sd_pipeline(prompt=prompt,num_images_per_prompt=1, num_inference_steps=num_inference_steps, guidance_scale=8.0, output_type='np' ).images

  img_clip_score = calculate_clip_score(image, prompt)
  total_clip_score += img_clip_score
  print(img_clip_score)

  i += 1

# clip score:
total_clip_score = total_clip_score/stop
print(total_clip_score)

# (6, 512, 512, 3)

<class 'numpy.ndarray'>
CLIP score: 36.0942


## Consistency Model

In [ ]:
import torch
from diffusers import ConsistencyModelPipeline

device = "cuda"
# Load the cd_bedroom256_lpips checkpoint.
model_id_or_path = "openai/diffusers-cd_bedroom256_lpips"
pipe = ConsistencyModelPipeline.from_pretrained(model_id_or_path, torch_dtype=torch.float16)
pipe.to(device)

# + pipe.unet = torch.compile(pipe.unet, mode="reduce-overhead", fullgraph=True)

# Multistep sampling
# Timesteps can be explicitly specified; the particular timesteps below are from the original GitHub repo:
# https://github.com/openai/consistency_models/blob/main/scripts/launch.sh#L83

# iterate thru 500 images and feed images to clipscore
for _ in range(10):
    image = pipe(timesteps=[17, 0]).images[0]
    image.show()

unet/diffusion_pytorch_model.safetensors not found
Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

Both `num_inference_steps`: 1 and `timesteps`: [17, 0] are supplied; `timesteps` will be used over `num_inference_steps`.
/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:139: FutureWarning: Accessing config attribute `sigma_min` directly via 'CMStochasticIterativeScheduler' object attribute is deprecated. Please access 'sigma_min' over 'CMStochasticIterativeScheduler's config object instead, e.g. 'scheduler.config.sigma_min'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


  0%|          | 0/2 [00:00<?, ?it/s]

Both `num_inference_steps`: 1 and `timesteps`: [17, 0] are supplied; `timesteps` will be used over `num_inference_steps`.
/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:139: FutureWarning: Accessing config attribute `sigma_min` directly via 'CMStochasticIterativeScheduler' object attribute is deprecated. Please access 'sigma_min' over 'CMStochasticIterativeScheduler's config object instead, e.g. 'scheduler.config.sigma_min'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


...

  0%|          | 0/2 [00:00<?, ?it/s]

Both `num_inference_steps`: 1 and `timesteps`: [17, 0] are supplied; `timesteps` will be used over `num_inference_steps`.
/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:139: FutureWarning: Accessing config attribute `sigma_min` directly via 'CMStochasticIterativeScheduler' object attribute is deprecated. Please access 'sigma_min' over 'CMStochasticIterativeScheduler's config object instead, e.g. 'scheduler.config.sigma_min'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


  0%|          | 0/2 [00:00<?, ?it/s]

Both `num_inference_steps`: 1 and `timesteps`: [17, 0] are supplied; `timesteps` will be used over `num_inference_steps`.
/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:139: FutureWarning: Accessing config attribute `sigma_min` directly via 'CMStochasticIterativeScheduler' object attribute is deprecated. Please access 'sigma_min' over 'CMStochasticIterativeScheduler's config object instead, e.g. 'scheduler.config.sigma_min'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import torch

from diffusers import ConsistencyModelPipeline

device = "cuda"
# Load the cd_imagenet64_l2 checkpoint.
model_id_or_path = "openai/diffusers-cd_imagenet64_l2"
pipe = ConsistencyModelPipeline.from_pretrained(model_id_or_path, torch_dtype=torch.float16)
pipe.to(device)

# Onestep Sampling
image = pipe(num_inference_steps=1).images[0]
image.save("cd_imagenet64_l2_onestep_sample.png")

# Onestep sampling, class-conditional image generation
# ImageNet-64 class label 145 corresponds to king penguins
image = pipe(num_inference_steps=1, class_labels=145).images[0]
image.save("cd_imagenet64_l2_onestep_sample_penguin.png")

# Multistep sampling, class-conditional image generation
# Timesteps can be explicitly specified; the particular timesteps below are from the original Github repo:
# https://github.com/openai/consistency_models/blob/main/scripts/launch.sh#L77
image = pipe(num_inference_steps=None, timesteps=[22, 0], class_labels=145).images[0]
image.save("cd_imagenet64_l2_multistep_sample_penguin.png")

model_index.json:   0%|          | 0.00/254 [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

unet/config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

...

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:139: FutureWarning: Accessing config attribute `sigma_min` directly via 'CMStochasticIterativeScheduler' object attribute is deprecated. Please access 'sigma_min' over 'CMStochasticIterativeScheduler's config object instead, e.g. 'scheduler.config.sigma_min'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

## Latent Consistency Model

In [ ]:
#LCM .
from diffusers import DiffusionPipeline, UNet2DConditionModel, LCMScheduler
import torch

unet = UNet2DConditionModel.from_pretrained(
    "latent-consistency/lcm-sdxl",
    torch_dtype=torch.float16,
    variant="fp16",
)
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", unet=unet, torch_dtype=torch.float16
).to("cuda")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

prompt = "Self-portrait oil painting, a beautiful cyborg with golden hair, 8k"

generator = torch.manual_seed(0)
image = pipe(
    prompt=prompt, num_inference_steps=4, generator=generator, guidance_scale=8.0
).images[0]

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


config.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/5.14G [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

The config attributes {'skip_prk_steps': True} were passed to LCMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


  0%|          | 0/4 [00:00<?, ?it/s]

## ClipScore Eval

In [ ]:
!git clone https://github.com/jmhessel/clipscore.git


Cloning into 'clipscore'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 116 (delta 50), reused 106 (delta 49), pack-reused 0
Receiving objects: 100% (116/116), 663.63 KiB | 21.41 MiB/s, done.
Resolving deltas: 100% (50/50), done.


--2024-02-18 21:20:47--  https://github.com/jmhessel/clipscore/tree/main/example
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2180 (2.1K) [text/plain]
Saving to: ‘example’

example             100%[===================>]   2.13K  --.-KB/s    in 0s      

2024-02-18 21:20:47 (29.2 MB/s) - ‘example’ saved [2180/2180]



In [ ]:
!ls -a


.					       clipscore.py
..					       .config
cd_imagenet64_l2_multistep_sample_penguin.png  example
cd_imagenet64_l2_onestep_sample_penguin.png    .ipynb_checkpoints
cd_imagenet64_l2_onestep_sample.png	       sample_data
clipscore


In [ ]:
!pip install clip

  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-0.2.0-py3-none-any.whl size=6989 sha256=a1e609d0bba973f139a681897a0b93335f077d2ed0975308ef4ac45865a20db9
  Stored in directory: /root/.cache/pip/wheels/7f/5c/e6/2c0fdb453a3569188864b17e9676bea8b3b7e160c037117869
Successfully built clip


In [ ]:
!pip install sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!ls -a


.   checksums	  example	    generation_eval_utils.py  LICENSE	 requirements.txt
..  clipscore.py  flickr8k_example  .git		      README.md


In [ ]:
%cd clipscore

/content/clipscore


In [ ]:
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

/bin/bash: line 1: conda: command not found
  Using cached ftfy-6.1.3-py3-none-any.whl (53 kB)
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-zfvm5awb
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-zfvm5awb
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=cfda7a840533e4096d5f4536b9d8b0acf2835ed7ec68cb3788080439e6f9ec46
  Stored in directory: /tmp/pip-ephem-wheel-cache-03y8gmvy/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [ ]:
!python clipscore.py example/good_captions.json example/images/

# where  example/good_captions.json looks like

100%|███████████████████████████████████████| 338M/338M [00:19<00:00, 18.2MiB/s]
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100% 1/1 [00:01<00:00,  1.74s/it]
100% 1/1 [00:01<00:00,  1.04s/it]
/content/clipscore/clipscore.py:153: UserWarning: due to a numerical instability, new numpy normalization is slightly different than paper results. to exactly replicate paper results, please use numpy version less than 1.21, e.g., 1.20.3.
  warnings.warn(
CLIPScore: 0.8613


In [ ]:
!pip uninstall clip

Found existing installation: clip 0.2.0
Uninstalling clip-0.2.0:
  Would remove:
    /usr/local/bin/clip
    /usr/local/lib/python3.10/dist-packages/clip-0.2.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/clip/*
Proceed (Y/n)? Y
  Successfully uninstalled clip-0.2.0


## clipscore (using hugging face)

In [ ]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 10.6 MB/s eta 0:00:00


In [ ]:
!pip install diffusers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.7 MB/s eta 0:00:00


In [ ]:
import torch
from diffusers import ConsistencyModelPipeline
device = "cuda"

# model_ckpt = "CompVis/stable-diffusion-v1-4" # TODO: replace w lcm
model_id_or_path = "openai/diffusers-cd_bedroom256_lpips"
pipe = ConsistencyModelPipeline.from_pretrained(model_id_or_path, torch_dtype=torch.float16)
pipe.to(device)

# sd_pipeline = StableDiffusionPipeline.from_pretrained(model_ckpt, torch_dtype=torch.float16).to("cuda")

unet/diffusion_pytorch_model.safetensors not found
Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

ConsistencyModelPipeline {
  "_class_name": "ConsistencyModelPipeline",
  "_diffusers_version": "0.26.3",
  "_name_or_path": "openai/diffusers-cd_bedroom256_lpips",
  "scheduler": [
    "diffusers",
    "CMStochasticIterativeScheduler"
  ],
  "unet": [
    "diffusers",
    "UNet2DModel"
  ]
}

In [ ]:
image = pipe(num_inference_steps=1).images[0]
image.save("cd_imagenet64_l2_onestep_sample.png")
# this works but i realize consistendcy models dont use a text input ?


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
from torchmetrics.functional.multimodal import clip_score
from functools import partial

clip_score_fn = partial(clip_score, model_name_or_path="openai/clip-vit-base-patch16")



# CLIP score: 35.7038

In [ ]:
prompts = [
    "a photo of an astronaut riding a horse on mars",
    "A high tech solarpunk utopia in the Amazon rainforest",
    "A pikachu fine dining with a view to the Eiffel Tower",
    "A mecha robot in a favela in expressionist style",
    "an insect robot preparing a delicious meal",
    "A small cabin on top of a snowy mountain in the style of Disney, artstation",
]

# images = cm_pipeline(prompts, num_images_per_prompt=1, output_type="np").images

# for _ in range(10):
for _ in range(10):
      image = pipe(timesteps=[17, 0]).images[0]
      image.show()



Both `num_inference_steps`: 1 and `timesteps`: [17, 0] are supplied; `timesteps` will be used over `num_inference_steps`.
/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:139: FutureWarning: Accessing config attribute `sigma_min` directly via 'CMStochasticIterativeScheduler' object attribute is deprecated. Please access 'sigma_min' over 'CMStochasticIterativeScheduler's config object instead, e.g. 'scheduler.config.sigma_min'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


  0%|          | 0/2 [00:00<?, ?it/s]

Both `num_inference_steps`: 1 and `timesteps`: [17, 0] are supplied; `timesteps` will be used over `num_inference_steps`.
/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:139: FutureWarning: Accessing config attribute `sigma_min` directly via 'CMStochasticIterativeScheduler' object attribute is deprecated. Please access 'sigma_min' over 'CMStochasticIterativeScheduler's config object instead, e.g. 'scheduler.config.sigma_min'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


  0%|          | 0/2 [00:00<?, ?it/s]

Both `num_inference_steps`: 1 and `timesteps`: [17, 0] are supplied; `timesteps` will be used over `num_inference_steps`.
/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:139: FutureWarning: Accessing config attribute `sigma_min` directly via 'CMStochasticIterativeScheduler' object attribute is deprecated. Please access 'sigma_min' over 'CMStochasticIterativeScheduler's config object instead, e.g. 'scheduler.config.sigma_min'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


...

  0%|          | 0/2 [00:00<?, ?it/s]

Both `num_inference_steps`: 1 and `timesteps`: [17, 0] are supplied; `timesteps` will be used over `num_inference_steps`.
/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:139: FutureWarning: Accessing config attribute `sigma_min` directly via 'CMStochasticIterativeScheduler' object attribute is deprecated. Please access 'sigma_min' over 'CMStochasticIterativeScheduler's config object instead, e.g. 'scheduler.config.sigma_min'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


  0%|          | 0/2 [00:00<?, ?it/s]

Both `num_inference_steps`: 1 and `timesteps`: [17, 0] are supplied; `timesteps` will be used over `num_inference_steps`.
/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:139: FutureWarning: Accessing config attribute `sigma_min` directly via 'CMStochasticIterativeScheduler' object attribute is deprecated. Please access 'sigma_min' over 'CMStochasticIterativeScheduler's config object instead, e.g. 'scheduler.config.sigma_min'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
for _ in range(10):
      image = pipe(timesteps=[17, 0]).images[0]
      image.show()

## using mini dataset (not used rn)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("sayakpaul/instructpix2pix-demo", split="train")
dataset.features

## load data

## Vae

In [ ]:
https://medium.com/@judyyes10/generate-images-using-variational-autoencoder-vae-4d429d9bdb5

## Latent Consistency Model

In [ ]:
!pip install diffusers datasets torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
from diffusers import DiffusionPipeline
import torch

pipe = DiffusionPipeline.from_pretrained("SimianLuo/LCM_Dreamshaper_v7")
# To save GPU memory, torch.float16 can be used, but it may compromise image quality.
pipe.to(torch_device="cpu", torch_dtype=torch.float32)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/588 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/diffusers/pipelines/pipeline_utils.py:781: FutureWarning: `torch_dtype` is deprecated and will be removed in version 0.27.0. 
  deprecate("torch_dtype", "0.27.0", "")
/usr/local/lib/python3.10/dist-packages/diffusers/pipelines/pipeline_utils.py:784: FutureWarning: `torch_device` is deprecated and will be removed in version 0.27.0. 
  deprecate("torch_device", "0.27.0", "")


LatentConsistencyModelPipeline {
  "_class_name": "LatentConsistencyModelPipeline",
  "_diffusers_version": "0.26.3",
  "_name_or_path": "SimianLuo/LCM_Dreamshaper_v7",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "LCMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
from torchmetrics.functional.multimodal import clip_score
from functools import partial
### dont move to another cell
clip_score_fn = partial(clip_score, model_name_or_path="openai/clip-vit-base-patch16")

def calculate_clip_score(images, prompts):
    images_int = (images * 255).astype("uint8")
    clip_score = clip_score_fn(torch.from_numpy(images_int).permute(0, 3, 1, 2), prompts).detach()
    return round(float(clip_score), 4)
### dont move to another cell

In [ ]:




prompt = "golden retriever running through the snow" # change this with data set prompt

# Can be set to 1~50 steps. LCM support fast inference even <= 4 steps. Recommend: 1~8 steps.
num_inference_steps = 1
image = pipe(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=8.0, output_type='np').images
print(image.shape)
# images[0].save("image.png")
# print(type(image))

lcm_clip_score = calculate_clip_score(image, prompt)
print(f"CLIP score: {lcm_clip_score}")
# clip score:

  0%|          | 0/1 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.


(1, 768, 768, 3)
CLIP score: 17.4362


In [ ]:

# for testing https://huggingface.co/spaces/taesiri/CLIPScore

# randomly sample 1000 captions from coco caption benchmark

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-creden

In [ ]:
# dont run this again it takes forever
from datasets import load_dataset

dataset = load_dataset("HuggingFaceM4/COCO", split='train[:1%]')

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for HuggingFaceM4/COCO contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/HuggingFaceM4/COCO
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [ ]:
dataset.shape

In [ ]:
dataset.features

In [ ]:
tok =dataset['sentences'][:10]
print(tok)
print(tok[0]['raw'])

In [ ]:
tokens = dataset['sentences'][:100]

prompts = []
for token in tokens:
   prompts.append(token['raw'])

In [ ]:
print(prompts)

In [ ]:
pipe.safety_checker = None

# Can be set to 1~50 steps. LCM support fast inference even <= 4 steps. Recommend: 1~8 steps.
total_clip_score = 0
for prompt in prompts:
  num_inference_steps = 1

  image = pipe(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=8.0, output_type='np' ).images

  img_clip_score = calculate_clip_score(image, prompt)
  total_clip_score += img_clip_score
  print(img_clip_score)

# clip score:
total_clip_score = total_clip_score/100
print(total_clip_score)

  0%|          | 0/1 [00:00<?, ?it/s]

27.0051


  0%|          | 0/1 [00:00<?, ?it/s]

26.8966


  0%|          | 0/1 [00:00<?, ?it/s]

...

25.9361


  0%|          | 0/1 [00:00<?, ?it/s]

23.9536


  0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Image Reward


In [ ]:
https://huggingface.co/THUDM/ImageReward


In [ ]:
!pip install image-reward



In [ ]:
mport os
import torch
import ImageReward as reward

if __name__ == "__main__":
    prompt = "a painting of an ocean with clouds and birds, day time, low depth field effect"
    img_prefix = "assets/images"
    generations = [f"{pic_id}.webp" for pic_id in range(1, 5)]
    img_list = [os.path.join(img_prefix, img) for img in generations]
    model = reward.load("ImageReward-v1.0")
    with torch.no_grad():
        _, rewards = model.inference_rank(prompt, img_list)
        # Print the result
        print("\nPreference predictions:\n")
        print(f"rewards = {rewards}")
        for index in range(len(img_list)):
            score = model.score(prompt, img_list[index])
            print(f"{generations[index]:>16s}: {score:.2f}")

## LCM - LORA

In [ ]:
# https://huggingface.co/latent-consistency/lcm-lora-sdxl

import torch
from diffusers import LCMScheduler, AutoPipelineForText2Image

model_id = "stabilityai/stable-diffusion-xl-base-1.0"
adapter_id = "latent-consistency/lcm-lora-sdxl"

sdxlpipe = AutoPipelineForText2Image.from_pretrained(model_id, torch_dtype=torch.float16, variant="fp16")
sdxlpipe.scheduler = LCMScheduler.from_config(sdxlpipe.scheduler.config)
sdxlpipe.to("cuda")

# load and fuse lcm lora
sdxlpipe.load_lora_weights(adapter_id)
sdxlpipe.fuse_lora()



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

...

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

The config attributes {'skip_prk_steps': True} were passed to LCMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


pytorch_lora_weights.safetensors:   0%|          | 0.00/394M [00:00<?, ?B/s]

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
/usr/local/lib/python3.10/dist-packages/diffusers/loaders/lora.py:1078: FutureWarning: `fuse_text_encoder_lora` is deprecated and will be removed in version 0.27. You are using an old version of LoRA backend. This will be deprecated in the next releases in favor of PEFT make sure to install the latest PEFT and transformers packages in the future.
  deprecate("fuse_text_encoder_lora", "0.27", LORA_DEPRECATION_MESSAGE)


In [ ]:

# Can be set to 1~50 steps. LCM support fast inference even <= 4 steps. Recommend: 1~8 steps.
total_clip_score = 0

endLoop = 10
sdxlpipe.safety_checker = None
for i in range(endLoop):
  prompt = prompt[i]
  num_inference_steps = 1
  image = sdxlpipe(prompt=prompt, num_inference_steps=num_inference_steps, guidance_scale=8.0, output_type='np').images

  img_clip_score = calculate_clip_score(image, prompt)
  total_clip_score += img_clip_score
  print(img_clip_score)

# clip score:
total_clip_score = total_clip_score/endLoop
print("score for sdxl+lora")
print(total_clip_score)